In [27]:
import codecs
import glob
import math
import os
from decimal import Decimal
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline


plt.rcParams["font.family"] = "Times New Roman"

data_dir = "data/input/OH exited state (fc)/alpha1/1A/*.out"
result_dir = data_dir.split("*.out")[0].replace("input", "output")
paths = [p for p in glob.glob(data_dir)]

excited_state_1 = "Excited State   1"
excited_state_2 = "Excited State   2"
excited_state_3 = "Excited State   3"

gs_high_model_key = "RB3LYP"
initial_key_1 = "Excited State   1"
end_key_1 = "SavETr"
initial_key_2 = "low   system:  model energy"
end_key_2 = "ONIOM: calculating electric field derivatives."
df_output = pd.DataFrame()

for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]
    torsion_angle = int(file_name.split("@")[0].split("_")[-1])

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
        # initial_key_1を含む行を探す
        initial_key_1_df = df[df['Col_1'].str.contains(initial_key_1, case=False)]
        initial_key_1_index = initial_key_1_df.index[0]
        df_1 = df.iloc[initial_key_1_index:]
        # initial_key_1を含む行以降のend_key_1を含む行を抜き出す
        end_key_1_df = df_1[df_1['Col_1'].str.contains(end_key_1)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_1_index = end_key_1_df.index[0] - initial_key_1_index
        df_1 = df_1.iloc[:end_key_1_index]
        # 該当行の抜き出しが完了
        
        S1_data = [a for a in df_1[df_1['Col_1'].str.contains(excited_state_1)]['Col_1'].iloc[0].split(' ') if a != '']
        f_1 = float(S1_data[-2].split('f=')[-1])
        E_1 = float(S1_data[4])
        nm_1 = float(S1_data[-4])
        
        S2_data = [a for a in df_1[df_1['Col_1'].str.contains(excited_state_2)]['Col_1'].iloc[0].split(' ') if a != '']
        f_2 = float(S2_data[-2].split('f=')[-1])
        E_2 = float(S2_data[4])
        nm_2 = float(S2_data[-4])
        
        S3_data = [a for a in df_1[df_1['Col_1'].str.contains(excited_state_3)]['Col_1'].iloc[0].split(' ') if a != '']
        f_3 = float(S3_data[-2].split('f=')[-1])
        E_3 = float(S3_data[4])
        nm_3 = float(S3_data[-4])
    
        data_list = [f_1, E_1, nm_1, f_2, E_2, nm_2, f_3, E_3, nm_3]
        df_parameters = pd.DataFrame(data=data_list, 
                            columns=[torsion_angle],
                            index=["f_1", "S1", "nm_1", "f_2", "S2", "nm_2", "f_3", "S3", "nm_3"]).T
        
        
        
        gs_high_model = float([a for a in df[df['Col_1'].str.contains("RB3LYP")]['Col_1'].iloc[0].split(' ') if a != ''][4])
        initial_key_2_df = df[df['Col_1'].str.contains(initial_key_2, case=False)]
        initial_key_2_index = initial_key_2_df.index[0]
        df_2 = df.iloc[initial_key_2_index:]
        # initial_key_2を含む行以降のend_keyを含む行を抜き出す
        end_key_2_df = df_2[df_2['Col_1'].str.contains(end_key_2)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_2_index = end_key_2_df.index[0] - initial_key_2_index
        df_2 = df_2.iloc[:end_key_2_index]
        # 該当行の抜き出しが完了
        
        # 各エネルギー値の抜き出し
        low_model = float(list(df_2['Col_1'])[0].split(":")[-1])
        high_model = float(list(df_2['Col_1'])[1].split(":")[-1])
        low_real = float(list(df_2['Col_1'])[2].split(":")[-1])
        high_real = float(list(df_2['Col_1'])[3].split("=")[-1])
        
        S1_high_real = gs_high_model+(E_1/27.2116) + low_real - low_model
        S2_high_real = gs_high_model+(E_2/27.2116) + low_real - low_model
        S3_high_real = gs_high_model+(E_3/27.2116) + low_real - low_model
        
        df_E = pd.DataFrame(data=[S1_high_real, S2_high_real, S3_high_real, high_real, low_real, high_model, low_model], 
                            columns=[torsion_angle],
                            index=["S1_high_real", "S2_high_real", "S3_high_real", "high real", "low real", "high model", "low model"]).T
        df_result = pd.concat([df_parameters, df_E], axis=1)
        
        df_output = pd.concat([df_output, df_result]).sort_index()

In [28]:
df_output

,f_1,S1,nm_1,f_2,S2,nm_2,f_3,S3,nm_3,S1_high_real,S2_high_real,S3_high_real,high real,low real,high model,low model
2,0.3138,3.6906,335.95,0.0965,4.0215,308.31,0.0172,4.5729,271.13,-9420.673869,-9420.661709,-9420.641446,-9420.673868,-9416.426256,-632.00931,-627.761698


In [29]:
os.makedirs(result_dir, exist_ok=True)
df_output.to_csv(result_dir + "result.csv")